## Setup

Import modules

In [240]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


from ydata_profiling import ProfileReport
import sweetviz as sv
from scipy.stats import pearsonr

Define constants and import dataset

In [241]:
DATASET = '../udataset/meat_production.csv'
START_DATE = '2010-01'
END_DATE = '2022-12'

meat_df = pd.read_csv(DATASET)
meat_df = meat_df.rename(columns = {'DateTime': 'year-month'})

In [242]:
UNEMPLOYMENTDATA = '../udataset/unemployment_by_state.csv'
unemployment_df = pd.read_csv(UNEMPLOYMENTDATA)
unemployment_df.columns = unemployment_df.columns.str.strip()
unemployment_df = unemployment_df.rename(columns={'YearMonth': 'year-month', 'unemploymentrate': 'unemployment-rate'})
unemployment_df.columns = unemployment_df.columns.str.lower()
unemployment_df = unemployment_df.rename(columns={'YearMonth': 'year-month', 'unemploymentrate': 'unemployment-rate'})
unemployment_df

,state,year-month,unemployment-rate
0,alabama,2010-01,11.7
1,alabama,2010-02,11.4
2,alabama,2010-03,11.0
3,alabama,2010-04,10.1
4,alabama,2010-05,9.9
...,...,...,...
8835,puerto rico,2023-10,5.5
8836,puerto rico,2023-11,5.7
8837,puerto rico,2023-12,5.8
8838,puerto rico,2024-01,5.8


In [243]:
POVERTYDATA ='../udataset/wealth_data_interpolated.csv'
poverty_df = pd.read_csv(POVERTYDATA)
poverty_df['date'] = poverty_df['date'].str.slice(0, 7)
poverty_df = poverty_df.rename(columns = {'date': 'year-month'})
poverty_df

,year-month,below_poverty_line_percent,state
0,2010-01,17.141096,alabama
1,2010-02,17.179452,alabama
2,2010-03,17.221918,alabama
3,2010-04,17.263014,alabama
4,2010-05,17.305479,alabama
...,...,...,...
7483,2021-08,10.700000,wyoming
7484,2021-09,10.700000,wyoming
7485,2021-10,10.700000,wyoming
7486,2021-11,10.700000,wyoming


In [244]:
unemployment_df = pd.merge(unemployment_df, poverty_df, on=['year-month', 'state'])
unemployment_df

,state,year-month,unemployment-rate,below_poverty_line_percent
0,alabama,2010-01,11.7,17.141096
1,alabama,2010-02,11.4,17.179452
2,alabama,2010-03,11.0,17.221918
3,alabama,2010-04,10.1,17.263014
4,alabama,2010-05,9.9,17.305479
...,...,...,...,...
7483,puerto rico,2021-08,9.2,42.347620
7484,puerto rico,2021-09,7.9,42.309405
7485,puerto rico,2021-10,7.1,42.271375
7486,puerto rico,2021-11,7.3,42.236039


In [245]:
#unemployment_rate = [9.63, 8.95, 8.07, 7.37, 6.17, 5.28, 4.87, 4.36, 3.90, 3.67, 8.05, 5.35, 3.61]
for state in unemployment_df['state'].unique():
    temp_df = unemployment_df[unemployment_df['state'] == state].copy()
    #temp_df['unemployment-rate'] = temp_df['below_poverty_line_percent']/temp_df['unemployment-rate']
    temp_df = temp_df.rename(columns={'unemployment-rate': f'{state}-unemployment-rate', 'below_poverty_line_percent': f"{state}-poverty-percent"})
    temp_df = temp_df.drop(columns=['state'])

    print(temp_df.columns)
    meat_df = pd.merge(meat_df, temp_df, on=['year-month'])


meat_df = meat_df.drop(columns = ['Unnamed: 0'])
#meat_df = meat_df[meat_df['year-month'].str.match(r'*-12')]
#print(new_df['unemployment_rate'].expanding().corr(new_df['S&P_growth']))

Index(['year-month', 'alabama-unemployment-rate', 'alabama-poverty-percent'], dtype='object')
Index(['year-month', 'alaska-unemployment-rate', 'alaska-poverty-percent'], dtype='object')
Index(['year-month', 'arizona-unemployment-rate', 'arizona-poverty-percent'], dtype='object')
Index(['year-month', 'arkansas-unemployment-rate', 'arkansas-poverty-percent'], dtype='object')
Index(['year-month', 'california-unemployment-rate',
       'california-poverty-percent'],
      dtype='object')
Index(['year-month', 'colorado-unemployment-rate', 'colorado-poverty-percent'], dtype='object')
Index(['year-month', 'connecticut-unemployment-rate',
       'connecticut-poverty-percent'],
      dtype='object')
Index(['year-month', 'delaware-unemployment-rate', 'delaware-poverty-percent'], dtype='object')
Index(['year-month', 'district of columbia-unemployment-rate',
       'district of columbia-poverty-percent'],
      dtype='object')
Index(['year-month', 'florida-unemployment-rate', 'florida-poverty-perc

In [246]:
meat_df.dtypes
#meat_df = meat_df[meat_df['year-month'].str.contains(r'-01')]
meat_df

,year-month,Beef,Veal,Pork,Lamb_and_mutton,Broilers,Other_chicken,Turkey,alabama-unemployment-rate,alabama-poverty-percent,...,washington-unemployment-rate,washington-poverty-percent,west virginia-unemployment-rate,west virginia-poverty-percent,wisconsin-unemployment-rate,wisconsin-poverty-percent,wyoming-unemployment-rate,wyoming-poverty-percent,puerto rico-unemployment-rate,puerto rico-poverty-percent
0,2021-12,2.326800e+09,5100000.0,2.393800e+09,10500000.0,3.700100e+09,44700000.0,453300000.0,2.3,15.700138,...,3.8,9.900138,3.5,16.800138,2.5,10.700000,3.5,10.700000,6.7,42.201100
1,2021-11,2.342800e+09,4500000.0,2.397900e+09,10900000.0,3.665300e+09,40600000.0,485500000.0,2.4,15.704911,...,3.9,9.904911,3.6,16.804911,2.4,10.700000,3.5,10.700000,7.3,42.236039
2,2021-10,2.324900e+09,4400000.0,2.360000e+09,10100000.0,3.827800e+09,44300000.0,470500000.0,2.7,15.710378,...,4.1,9.910378,3.6,16.810378,2.5,10.700000,3.3,10.700000,7.1,42.271375
3,2021-09,2.259700e+09,4300000.0,2.253600e+09,9200000.0,3.928500e+09,45800000.0,474200000.0,2.9,15.716847,...,4.6,9.916847,3.9,16.816847,2.9,10.700000,3.3,10.700000,7.9,42.309405
4,2021-08,2.324000e+09,4200000.0,2.201200e+09,8800000.0,3.902000e+09,47000000.0,458600000.0,3.5,15.723843,...,5.1,9.923843,4.9,16.823843,3.7,10.700000,3.9,10.700000,9.2,42.347620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2010-05,2.060700e+09,9800000.0,1.608700e+09,11800000.0,3.020000e+09,40200000.0,455100000.0,9.9,17.305479,...,8.9,12.264384,8.2,17.441096,8.3,11.752833,7.0,9.865588,17.1,45.272433
140,2010-04,2.110800e+09,10800000.0,1.835900e+09,12300000.0,3.038300e+09,40700000.0,490000000.0,10.1,17.263014,...,8.9,12.230411,8.5,17.432603,8.8,11.719822,7.4,9.842634,17.2,45.261981
141,2010-03,2.180700e+09,11900000.0,2.025100e+09,17000000.0,3.162000e+09,43400000.0,425400000.0,11.0,17.221918,...,9.8,12.197534,9.6,17.424384,10.1,11.688565,8.1,9.824580,16.3,45.249793
142,2010-02,1.927200e+09,10500000.0,1.745000e+09,11900000.0,2.739500e+09,34700000.0,424200000.0,11.4,17.179452,...,10.1,12.163562,10.4,17.415890,10.4,11.656997,8.3,9.810761,16.1,45.234877


In [247]:
#print(meat_df[meat_df.columns[1:]].corr())
meat_df[meat_df.columns[1:]].corr().to_csv('../udataset/state_correlation.csv')
# meat_df['expanding_correlation'] = meat_df['delaware-poverty-percent'].expanding().corr(meat_df['Beef'])
# #meat_df['expanding_correlation'].to_csv('../udataset/state-correlation.csv')

# plt.figure(figsize=(10, 5))
# plt.plot(meat_df['year-month'], meat_df['expanding_correlation'], label='Cali')
# plt.xlabel('Year')
# plt.ylabel('Below Poverty Line (%)')
# plt.legend()
# plt.grid(True)
# plt.show()
#x = meat_df['expanding_correlation']
# plt.plot()

In [248]:
meat_df.to_csv('../udataset/meat_poverty_data.csv')

In [249]:
p_values = []
p_values_df = pd.DataFrame({'columns': meat_df.columns[1:]})
#r_values = []

for col in meat_df.columns[1:]:

    p_values_df[col] = np.nan
    p_values = []

    for col2 in meat_df.columns[1:]:
        stats = pearsonr(meat_df[pd.notna(meat_df[col])][col], meat_df[pd.notna(meat_df[col])][col2])
        p_values.append(stats[1])

    p_values_df[col] = p_values
    print(p_values)

p_values_df.to_csv('../udataset/p_values_state_poverty.csv')

#p_value_df['p_value'] = p_values
#p_value_df['r_value'] = r_values

#p_value_df

[0.0, 0.25224318227824033, 3.7489902353340255e-13, 0.7825758219852973, 1.9502659241039134e-17, 1.327474747465276e-13, 0.23106677634731815, 0.0003545108007239188, 4.4239581559238004e-12, 0.0013680542966677644, 0.020092440608894358, 0.11836481453058696, 2.6130949782018237e-12, 0.009694384745779296, 1.5804455571925386e-11, 0.37537304428225576, 1.3187166943072515e-12, 0.5451085311571869, 5.498161967947041e-09, 0.3953312165012569, 5.800359628594789e-06, 0.17345911793064, 1.7775741528989361e-06, 0.31612140961991736, 1.7777449664119814e-08, 0.14322401117405195, 4.3039004919515174e-12, 0.005144841602651546, 2.441316171207893e-12, 0.763400200162807, 1.5372219191997442e-10, 0.017576826633138903, 1.9595596722562576e-11, 0.026905464912740297, 2.5959421376497823e-12, 0.01192266434354212, 7.807363366633733e-12, 0.0024890161786992825, 5.450013272818001e-12, 0.016464645743322945, 2.5358816400941737e-11, 0.0012089632278640561, 7.552592177673078e-12, 0.3130817910547927, 3.2611639490665423e-07, 0.0014142

[3.7489902353340255e-13, 2.0737997595602863e-14, 0.0, 0.0014691252902655928, 4.268558765810969e-28, 0.007806156276430829, 0.21948447509161317, 3.807238359466288e-21, 4.021553125308138e-18, 0.0011693819692859782, 4.796897468028369e-16, 3.697669339199578e-14, 1.0106713224716053e-16, 3.524552603160619e-16, 3.168572509595228e-23, 4.770700996312374e-09, 1.5712692262867597e-16, 7.114915566054522e-09, 2.6383903158712676e-27, 6.53931899949166e-10, 0.5805882245935504, 4.212625660440511e-09, 0.4067142441871503, 2.3940435305574622e-14, 1.3834467085750894e-28, 2.7956608049957402e-12, 1.4767568806572854e-14, 2.700035232518849e-21, 2.9764132040882532e-18, 0.43005615950960124, 1.9615800336251967e-15, 5.420907295674871e-12, 5.636896877985579e-20, 2.441783442432535e-10, 3.0771535725882914e-16, 5.91000998223912e-13, 2.832625594513027e-16, 1.4899477990294222e-09, 1.1349959762095725e-18, 2.940122184058807e-13, 1.9859894402881404e-18, 1.045522742066336e-18, 2.393385015897308e-19, 2.539053980882192e-05, 0.5

/tmp/ipykernel_53944/4210159914.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  p_values_df[col] = np.nan


[0.03691685582388944, 9.638065520727733e-14, 2.877669033047371e-10, 0.009093631262017, 1.666048881840124e-08, 0.34484644612017984, 0.05488556600847332, 7.974106809105711e-56, 0.05025091801672521, 1.4339372651694178e-38, 3.8528288728598398e-22, 3.7897892789433617e-76, 0.09837768159599876, 1.2370942619400655e-61, 0.00123044152070813, 5.0617039151762304e-79, 0.08124010349108583, 2.9823850366673754e-54, 3.7561299655935735e-07, 4.262580095169857e-65, 0.000639368611949825, 5.759659964400358e-80, 1.7883959980289878e-07, 1.5695690226396243e-52, 1.3205259634690736e-07, 1.6088043970637273e-88, 0.18002560622516695, 1.2521323779779878e-53, 0.018337483911639554, 4.185064386801398e-34, 0.003982388070129531, 4.785119086029857e-50, 0.018466431742260103, 5.08872715248739e-94, 0.10987785986122578, 6.420449811603186e-76, 0.09240409577379871, 5.45126970091538e-70, 0.03237168277555048, 9.683495551097222e-83, 0.011754635660990382, 1.0118880508093829e-45, 0.032243649729852095, 6.310530273417662e-49, 8.359489

/tmp/ipykernel_53944/4210159914.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  p_values_df[col] = np.nan


...

[7.1336219762715815e-06, 0.009375564410776308, 0.4814806050965808, 0.7786809781128912, 0.5028458518629906, 0.37577642467602757, 0.08985542867797155, 0.0007138674962129212, 9.968905859634273e-16, 2.2733385238213384e-05, 0.00019132881939999777, 1.0617655209325653e-06, 1.9153112151629993e-17, 0.002150302393914601, 1.0797824525527376e-09, 4.926217366550614e-08, 1.1221291893603666e-18, 2.4128077450576596e-08, 3.018019023293685e-05, 1.1717296651039382e-05, 2.9899762128352547e-69, 2.607760676444481e-07, 2.1005212011276517e-43, 8.257372444289797e-06, 0.003193070222391475, 9.454539120088135e-07, 8.320531923478933e-23, 0.0008204475797120306, 3.878322170926024e-16, 8.440960983421135e-06, 1.4059342392889626e-18, 1.843991191466662e-06, 2.0788794951579232e-12, 0.0002516141009430114, 1.0297552223388756e-19, 1.5881779677263614e-05, 5.320632519270381e-19, 9.810974753170116e-06, 4.738445303394579e-16, 6.409067457196875e-06, 7.965920930116474e-17, 1.74496555645804e-05, 1.03611078732485e-12, 0.00162745675

/tmp/ipykernel_53944/4210159914.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  p_values_df[col] = np.nan


[0.0001756397710650853, 1.858844429396054e-48, 5.567875948573649e-27, 3.7964743323065186e-09, 1.3127953670209375e-28, 0.012649460932311115, 0.4878753288169268, 4.379574613207314e-37, 6.6445060761171756e-21, 2.7344325070163053e-05, 1.9284895927882869e-41, 7.860132193392845e-24, 7.218365354679095e-19, 7.878591629064335e-26, 3.989220635504964e-30, 6.15843756406708e-17, 1.7108747076391168e-18, 2.554950311322443e-16, 1.4728658991251367e-45, 3.5822899217790083e-17, 0.2942012322380579, 1.5463010162466887e-12, 0.343555926703412, 1.6105789589822209e-27, 5.751300056672958e-51, 4.752495815377402e-22, 1.3866434357183586e-15, 4.820751888179561e-41, 1.0911974875835722e-21, 0.23701634090663284, 1.2924219614241141e-18, 1.8642778727553555e-24, 1.6031137965581413e-24, 8.914997250048738e-16, 7.368935678700508e-18, 4.937257715769221e-21, 2.9704909227377287e-18, 6.418316183343502e-14, 1.4524291507639597e-21, 2.20414126517589e-17, 1.895961262798853e-21, 3.809603355741981e-26, 2.5135913536346046e-23, 1.51402

/tmp/ipykernel_53944/4210159914.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  p_values_df[col] = np.nan


[2.0708183401201558e-10, 9.476766200888166e-23, 6.10874170630665e-23, 2.4562613198004326e-12, 1.98865738262021e-24, 0.029343523528379555, 0.3575959519911017, 4.6639020959663456e-14, 7.501057564210126e-57, 0.2192407073911348, 4.4042691924925486e-13, 4.1942124617850714e-06, 1.5647374695760498e-56, 2.1741585753614663e-07, 2.171709932750964e-65, 0.021040328473015034, 6.472995067825222e-50, 0.049875604248857834, 1.2975505427254816e-56, 0.012634598403380909, 0.01847546618547158, 0.05838105241068427, 1.7528292856201973e-07, 1.0467731495004673e-05, 7.14804724134912e-69, 5.28544792414977e-05, 6.138924159649202e-42, 2.657831663909295e-13, 5.247860561569528e-54, 0.0012213087547357287, 1.8592529580679687e-28, 3.983695074860806e-06, 1.4770627236430868e-71, 0.004169927069081845, 5.061799255230993e-47, 1.5312435202098004e-05, 1.0791029947225098e-50, 0.007133988120569442, 1.5557772708039174e-51, 6.774744818235492e-05, 9.618818704321524e-43, 5.968038524472309e-08, 5.0215413771914655e-71, 0.332442570576

In [250]:
#new_df.to_csv('../udataset/growth_vs_unemployment_month.csv')